In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [2]:
# Option 1: 
# Use a CW equal to the 3 panels, and replicate 3 times along the row.
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'
epwfile = r'C:\Users\sayala\Documents\RadianceScenes\Demo3\EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw'
# This is the only thing that changes for running on the HPC at the moment
#testfolder = '/scratch/sayala/RadianceScenes/Test1'
#epwfile='/scratch/sayala/EPWs/USA_VA_Richmond.Intl.AP.724010_TMY.epw'

simulationname = 'TestHPC1'
moduletype = 'Longi'

# Control Variables
cumulativesky=False
rewriteModule=True
hpc=True

# Scene Variables 
gcr = 0.35   # ground cover ratio,  = module_height / pitch
albedo = 0.3     # ground albedo
nMods = 20  # replicating the unit 3 times
nRows = 7  # only 1 row

# Analysis Variabls
sensorsy = 210  # this will give 70 sensors per module.
modWanted = nMods/2
rowWanted = nRows/2

# Fixed tilt variables
azimuth_ang=180 # Facing south
tilt =20 # tilt. 

# Tracker geometry options
backtrack = True
hub_height = 2   # tracker height at 0 tilt in meters (hub height)
limit_angle = 45 # tracker rotation limit angle
axis_azimuth=180
angledelta=5


# ModuleDict Variables
numpanels = 1
x=0.984  
y = 1.996
bifi=0.9
xgap = 0.01 # 10 centimeters.
ygap = 0.0
zgap = 0.1

# TorqueTube Parameters
torqueTube = True
axisofrotationTorqueTube=False
diameter=0.1
tubetype='Round'
torqueTubeMaterial='Metal_Grey'

# Cell Level Module Parameters
cellLevelModule = False
numcellsx=10
numcellsy=6
xcell=0.156
ycell=0.156
xcellgap=0.02
ycellgap=0.02


In [3]:
demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=bifi, 
                       torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                       material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                       rewriteModulefile = True, xgap=xgap, 
                       axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                       numcellsx=numcellsx, numcellsy = numcellsy)
sceneDict = {'module_type':moduletype, 'pitch': round(moduleDict['sceney'] / gcr,3),'height':hub_height, 'nMods':nMods, 'nRows':nRows} #, 'nMods':20, 'nRows':7}  

#epwfile = demo.getEPW(35.0375,-106.6055) # pull EPW data for CO Boulder
metdata = demo.readEPW(epwfile=epwfile, hpc=True, daydate='02_24') # read in the EPW weather data from above
trackerdict = demo.set1axis(cumulativesky=cumulativesky, axis_azimuth=axis_azimuth, limit_angle=limit_angle, angledelta=angledelta, backtrack=backtrack, gcr=gcr)
trackerdict = demo.gendaylit1axis(hpc=hpc)  #benchmark time: gendaylit2:105s.  gendaylit: 5s

# SAnity Check of SUnrise and Sunset Hours all YeaR:
#metdata = demo.readEPW(epwfile=epwfile) # read in the EPW weather data from above
#trackerdict = demo.set1axis(cumulativesky=cumulativesky, axis_azimuth=axis_azimuth, limit_angle=limit_angle, angledelta=angledelta, backtrack=backtrack, gcr=gcr)
#trackerdict = demo.gendaylit1axis()  #benchmark time: gendaylit2:105s.  gendaylit: 5s


path = C:\Users\sayala\Documents\RadianceScenes\Demo3
('\nModule Name:', 'Longi')
REWRITING pre-existing module file. 
Module Longi successfully created


C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:424: RuntimeWarning: invalid value encountered in minimum
  temp = np.minimum(axes_distance*cosd(wid), 1)
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:431: RuntimeWarning: invalid value encountered in less
  tracker_theta = np.where(wid < 0, wid + wc, wid - wc)
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:435: RuntimeWarning: invalid value encountered in greater
  tracker_theta[tracker_theta > max_angle] = max_angle
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:436: RuntimeWarning: invalid value encountered in less
  tracker_theta[tracker_theta < -max_angle] = -max_angle
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:543: RuntimeWarning: invalid value encountered in less
  surface_azimuth[surface_azimuth < 0] += 360
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:544: RuntimeWarning: invalid value encountered in greater_equal
  surface_azimuth[surface_a

Creating ~4000 skyfiles.  Takes 1-2 minutes
Adjusting solarposition for sunrise hour, 0 timeindex
Original datetime 1971-02-24 07:00:00-05:00
Localized and adjusted datetime 1971-02-24 06:54:00-05:00 

Adjusting solarposition for sunset hour, 11 timeindex
Original datetime 1971-02-24 18:00:00-05:00
Localized and adjusted datetime 1971-02-24 17:29:00-05:00 

Created 12 skyfiles in /skies/


In [4]:
trackerdict = demo.makeScene1axis(trackerdict=trackerdict, moduletype=moduletype, sceneDict=sceneDict, cumulativesky=cumulativesky) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis(trackerdict=trackerdict, hpc=True)



Making ~4000 .rad files for gendaylit 1-axis workflow (this takes a minute..)


IOError: [Errno 2] No such file or directory: '/scratch/sayala/RadianceScenes/Test1\\objects\\1axis02_24_10_1.294_5.703_20x7.rad'

In [ ]:
trackerdict = demo.analysis1axis(trackerdict, sceneDict=sceneDict, modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)


In [ ]:
import os
radfile = os.path.join(testfolder,'objects','%s_%0.5s_%0.5s_%sx%s.rad'%('sasdf',2.3,3, 7, 4) ) # update in 0.2.3 to shorten radnames
radfile

In [ ]:
radfile

In [ ]:
trackerdict['05_11_06']



In [ ]:
import pandas as pd
import pytz
import pvlib

lat = metdata.latitude
lon=metdata.longitude
elev=metdata.elevation
datessstime=metdata.datetime[1134]
datetime = pd.to_datetime(datessstime)
tz = metdata.timezone
try:  # make sure the data is tz-localized.
    datetimetz = datetime.tz_localize(pytz.FixedOffset(tz*60))  # either use pytz.FixedOffset (in minutes) or 'Etc/GMT+5'
except:  # data is tz-localized already. Just put it in local time.
    datetimetz = datetime.tz_convert(pytz.FixedOffset(tz*60))  

print datetime

solpos = pvlib.irradiance.solarposition.get_solarposition(datetimetz,lat,lon,elev)
solpos = pvlib.irradiance.solarposition.get_solarposition(datetimetz-pd.Timedelta(minutes = 30),lat,lon,elev)
print solpos

axis_tilt=0
axis_azimuth=180
limit_angle=45
backtrack=True
trackingdata = pvlib.tracking.singleaxis(solpos['zenith'], solpos['azimuth'], axis_tilt, axis_azimuth, limit_angle, backtrack, gcr)

self.tracker_theta = trackingdata['tracker_theta'].tolist()

print trackingdata
metdata.datetime.hour
    

In [ ]:
datetime.hour